In [1]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.environment import Environment
from azureml.core import Experiment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig


Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.44.0 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('azureml-core~=1.43.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.44.0 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('azureml-core~=1.43.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.44.0 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('azureml-core~=1.43.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = 

In [2]:
#ws=Workspace.create(name='hyperdive12',subscription_id='1db33695-8135-4616-9bb4-9574b401d454',resource_group='defaultsrg02',location='eastus2')
#auth=InteractiveLoginAuthentication(tenant_id='6ad91895-de06-485e-bc51-fce126cc8530'))
ws=Workspace.from_config('C:/Users/satya/Downloads/config/config.json')

In [3]:
register_data=Datastore.register_azure_blob_container(ws,datastore_name='adultds',container_name='adults',account_name='adultincomestor01',account_key='c16xclwCZDq/aQR9ClI7Q3brJtgDve733tbvk2wg3dimO88Q2DXD4+tcSwgz2CdXtXG0UQY4iK+I+ASt2gLxAA==')

In [4]:
datastore=Datastore.get(ws,'adultds')
path=[(datastore,'adultincome+trunc.csv')]
dataset=Dataset.Tabular.from_delimited_files(path)
dataset=dataset.register(ws,dataset,'adult_dataset')

In [5]:
dataset.to_pandas_dataframe().head()

,age,wc,education,marital status,race,gender,hours per week,IncomeClass
0,38,Private,HS-grad,Divorced,White,Male,40,<=50K
1,28,Private,Bachelors,Married,Black,Female,40,<=50K
2,37,Private,Masters,Married,White,Female,40,<=50K
3,31,Private,Masters,Never-married,White,Female,50,>50K
4,42,Private,Bachelors,Married,White,Male,40,>50K


In [6]:
cluster_name='cluster1'
config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D1',max_nodes=2)
cluster=AmlCompute.create(ws,cluster_name,config)

In [10]:
env=Environment(name='env1')
envdep=CondaDependencies.create(conda_packages=['scikit-learn','pandas','matplotlib','pip'],pip_packages=['azureml-interpret','azureml-dataset-runtime'])
env.python.conda_dependencies=envdep
env.register(ws)


{
    "assetId": "azureml://locations/eastus2/workspaces/0b311f32-5824-4709-95b0-a3e68d4b066e/environments/env1/versions/5",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220708.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "env1",
    "python": {
        "

In [12]:
script=ScriptRunConfig(source_directory='',script='model.py',environment=env,compute_target=cluster,arguments=['--input_dataset',dataset.as_named_input('raw_data')])
exp=Experiment(ws,'explainer_experiment')
run=exp.submit(script)
run.wait_for_completion(show_output=True)

RunId: explainer_experiment_1662299880_89ebaa03
Web View: https://ml.azure.com/runs/explainer_experiment_1662299880_89ebaa03?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/defaultsrg02/workspaces/hyperdive12&tid=6ad91895-de06-485e-bc51-fce126cc8530

Execution Summary
RunId: explainer_experiment_1662299880_89ebaa03
Web View: https://ml.azure.com/runs/explainer_experiment_1662299880_89ebaa03?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/defaultsrg02/workspaces/hyperdive12&tid=6ad91895-de06-485e-bc51-fce126cc8530



{'runId': 'explainer_experiment_1662299880_89ebaa03',
 'target': 'cluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-09-04T13:58:15.182709Z',
 'endTimeUtc': '2022-09-04T14:01:38.280264Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '62bee18a-7cab-4db6-b04e-e7a95eeed67b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '1342b4d4-b747-48c9-8094-825e3b786b4c'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'raw_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'model.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input_dataset', 'DatasetConsumptionConfig:raw_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cluster1',
  'dataReferences': {},
  'data': {'raw_data': {'dataLocation': {'dataset': {'id': '1